# Initialization

Test notebook for the C-MAPPS benchmark. Test different MLP architectures. 

First we import the necessary packages and create the global variables.

In [1]:
import math
import numpy as np
import csv
import copy
from scipy import stats
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import custom_scores
from data_handler_CMAPS import CMAPSDataHandler
from tunable_model import SequenceTunableModelRegression
import CMAPSAuxFunctions

from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Reshape, Conv2D, Flatten, MaxPooling2D,CuDNNLSTM
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras import backend as K
from keras import regularizers
from keras.layers import LSTM

from keras.layers.noise import GaussianNoise

import tensorflow as tf

Using TensorFlow backend.


# Define architectures

Define each one of the different architectures to be tested.

In [2]:
K.clear_session()  #Clear the previous tensorflow graph

l2_lambda_regularization = 0.20
l1_lambda_regularization = 0.10

def RULmodel_LSTM(input_shape):
    """Define the RNN model"""
    
    #Create a sequential model
    model = Sequential()
#     #model.add(Masking(mask_value=0, imput))
#     #model.add(LSTM(input_shape=input_shape, units=100, return_sequences=True, name='lstm1')))
#     model.add(LSTM(input_shape=input_shape, units=150, dropout=0.2, recurrent_dropout=0.2, return_sequences=False, name='lstm1'))
# #     model.add(Dense(20, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
# #                     kernel_regularizer=regularizers.l2(l2_lambda_regularization), name='fc1'))
#     model.add(Dense(1, activation='linear', kernel_initializer='glorot_normal', name='out'))
    model.add(CuDNNLSTM(input_shape=input_shape, units=32, return_sequences=True, name='lstm1'))
    model.add(Dropout(0.1))
    model.add(CuDNNLSTM(input_shape=input_shape, units=32, return_sequences=False, name='lstm2'))
    model.add(Dropout(0.1))
    #model.add(CuDNNLSTM(units=20, return_sequences=False, name='lstm2'))
#     model.add(Dropout(0.1))
    #16,8 yes
    model.add(Dense(16, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal',
                    name='fc1'))
    model.add(Dropout(0.1))

    model.add(Dense(8, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    name='fc2'))
    model.add(Dropout(0.1))

    model.add(Dense(1, activation='linear', kernel_initializer='glorot_normal', name='out'))
    
    return model

# def RULmodel_SN_4(input_shape):
#     #Create a sequential model
#     model = Sequential()
# #     model.add(GaussianNoise( 0.02 ))
#     model.add(Dropout(0.2))
#     #Add the layers for the model
# #     model.add(Dense(20, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
# #                     kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), 
# #                     name='fc1'))
# #     model.add(Dense(20, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
# #                     kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), 
# #                     name='fc2'))
# #     model.add(Dense(1, activation='linear', kernel_initializer='glorot_normal', 
# #                     kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), name='out'))

#     model.add(Dense(20, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
#                     kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), 
#                     name='fc1'))
#     #model.add(Dropout(0.2))
#     model.add(Dense(20, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
#                     kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), 
#                     name='fc2'))
    
# #     model.add(Dense(20, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
# #                     kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), 
# #                     name='fc3'))
    
#     model.add(Dense(1, activation='linear', kernel_initializer='glorot_normal', 
#                     kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), name='out'))
    
#     return model

In [3]:
def RULmodel_SN_1(input_shape):
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(250, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), 
                    name='fc1'))
    model.add(Dense(50, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), 
                    name='fc2'))
    model.add(Dense(1, activation='linear', 
                    kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), name='out'))
    
    return model

def RULmodel_SN_2(input_shape):
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(100, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), 
                    name='fc1'))
    model.add(Dense(50, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), 
                    name='fc2'))
    model.add(Dense(1, activation='linear', 
                    kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), name='out'))
    
    return model

def RULmodel_SN_3(input_shape):
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(50, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), 
                    name='fc1'))
    model.add(Dense(20, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), 
                    name='fc2'))
    model.add(Dense(1, activation='linear', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), name='out'))
    
    return model

def RULmodel_SN_5(input_shape):
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(50, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), 
                    name='fc1'))
    model.add(Dense(1, activation='linear', 
                    kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), name='out'))
    
    return model

def RULmodel_SN_6(input_shape):
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(50, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), 
                    name='fc1'))
    model.add(Dense(1, activation='linear', 
                    kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), name='out'))
    
    return model

In [4]:
def get_compiled_model(model_def, shape, model_type='ann'):
    
    K.clear_session()

    #Shared parameters for the models
    optimizer = Adam(lr=0.001 )#,beta_1=0.5
    lossFunction = "mean_squared_error"
    metrics = ["mse"]
    model = None

    #Create and compile the models

    if model_type=='ann':
        model = model_def(shape)
        model.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)
    elif model_type=='lstm':
        model = RULmodel_LSTM(shape)
        model.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)
    else:
        pass

    return model

In [5]:
#Define the usable models for this notebook

models = {'lstm-20-10':RULmodel_LSTM}
#models = {'shallow-16-16-12': RULmodel_SN_4}

#models = {'shallow-250-100':RULmodel_SN_4, 'shallow-100-50':RULmodel_SN_1, 'shallow-50-20':RULmodel_SN_2,
#          'shallow-20-20':RULmodel_SN_3, 'shallow-20':RULmodel_SN_5, 'shallow-10':RULmodel_SN_6}

# Process Data

In [6]:
features = ['T2', 'T24', 'T30', 'T50', 'P2', 'P15', 'P30', 'Nf', 'Nc', 'epr', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'farB', 'htBleed', 'Nf_dmd', 'PCNfR_dmd', 'W31', 'W32']
selected_indices = np.array([2, 3, 4, 7, 8, 9, 11, 12, 13, 14, 15, 17, 20, 21])
selected_features = list(features[i] for i in selected_indices-1)
data_folder = '../CMAPSSData'

window_size = 30
window_stride = 1
max_rul = 125

dHandler_cmaps = CMAPSDataHandler(data_folder, 1, selected_features, max_rul, 
                                  window_size, window_stride)

scaler = MinMaxScaler(feature_range=(-1, 1))
scaler = StandardScaler()



# Load Data

In [7]:
#For LSTM
#tModel.data_handler.data_scaler = scaler
#tModel.data_scaler = None

#For ANN
# tModel.data_handler.data_scaler = None
# tModel.data_scaler = scaler

# tModel.data_handler.sequence_length = 24
# tModel.data_handler.sequence_length = maxWindowSize[datasetNumber]
# tModel.data_handler.sequence_stride = 1
# tModel.data_handler.max_rul = 129

# tModel.load_data(unroll=True, verbose=1, cross_validation_ratio=0.3)
# tModel.print_data()

# Test on dataset 1

In [8]:
cv = 0.1

iterations = 10
lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)
num_features = len(selected_features)

windowSize = 30
#tModel.data_handler.sequence_length = maxWindowSize[datasetNumber]
windowStride = 1
constRul = 125

file = open("results/LSTM/10run32-32-1681.csv", "w")
csvfile = csv.writer(file, lineterminator='\n')

tModel = SequenceTunableModelRegression('lstm-20-10', None, lib_type='keras', data_handler=dHandler_cmaps)
tModel.epochs = 100

#For LSTM
tModel.data_handler.data_scaler = scaler
tModel.data_scaler = None

#For ANN
# tModel.data_handler.data_scaler = None
# #tModel.data_scaler = min_max_scaler
# tModel.data_scaler = scaler

for key, model_def in models.items():
  
    print("For model "+str(key))
    #file.write("For model "+str(key)+'\n\n')
  
    for i in range(1,2):

        dataset = i
        print("Computing for dataset "+str(i))
        #file.write("Computing for dataset "+str(i)+'\n\n')
      
        tempScoresRMSE = np.zeros((iterations,1))
        tempScoresRHS = np.zeros((iterations,1))
        tempTime = np.zeros((iterations,1))
        
        cv_tempScoresRMSE = np.zeros((iterations,1))
        cv_tempScoresRHS = np.zeros((iterations,1))
        cv_tempTime = np.zeros((iterations,1))
      
        tModel.data_handler.change_dataset(i)
        tModel.data_handler.sequence_length = windowSize
        tModel.data_handler.sequence_stride = windowStride
        tModel.data_handler.max_rul = constRul
        
        for d in range (6):
            cv = cv +0.1
        #tModel.load_data(unroll=True, verbose=0, cross_validation_ratio=0.2)
        #for lstm
         #For RNN
            

            for j in range(iterations):

                tModel.load_data(unroll=False, verbose=1, cross_validation_ratio=cv)
            #tModel.print_data()
                #csvfile.writerow()

            #input_shape = tModel.data_handler.sequence_length*num_features #For simple ANN
                input_shape = (tModel.data_handler.sequence_length,num_features)

                #Model needs to be recompiled everytime since they are different runs so weights should be reinit
                #model = get_compiled_model(model_def, input_shape, model_type='ann')
                model = get_compiled_model(model_def, input_shape, model_type='lstm')

                tModel.change_model(key, model, 'keras')
                #tModel.train_model(verbose=0, learningRate_scheduler=lrate)
                tModel.train_model(verbose=0) #lstm
                print("CV set")
                tModel.evaluate_model(['rhs', 'rmse'], round=2, cross_validation=True)

                cScores = tModel.scores
                rmse = math.sqrt(cScores['score_1'])
                rmse2 = cScores['rmse']
                rhs = cScores['rhs']
                time = tModel.train_time

                cv_tempScoresRMSE[j] = rmse2
                cv_tempScoresRHS[j] = rhs
                cv_tempTime[j] = time

                k = range(1,len(tModel.y_crossVal)+1)


                for k, rul_prediction, rul_prediction_rounded, true_rul in zip(k, np.ravel(tModel.y_predicted), tModel.y_predicted_rounded, np.ravel(tModel.y_crossVal)):
                    p = '{}, {}'.format(rul_prediction_rounded, true_rul)
                    print(p)
                    csvfile.writerow([p])
                #     p = "Engine {},Predicted RUL {},Rounded RUL {},Real RUL {}".format(i, rul_prediction, 
                #                                                                     rul_prediction_rounded, 
                #                                                                     true_rul)
                #     print(p)
                #     csvfile.writerow(["Engine {} Predicted RUL {} Rounded RUL {} Real RUL {}".format(i,rul_prediction, 
                #                                                                     rul_prediction_rounded, 
                #                                                                     true_rul)])
                print(cScores)
                #print("RMSE: {}".format(rmse))
                print("RMSE2: {}".format(cv_tempScoresRMSE[j]))
                print("RHS: {}".format(cv_tempScoresRHS[j]))
                print("Time : {} seconds".format(cv_tempTime[j]))

                csvfile.writerow(["RMSE2: {}".format(cv_tempScoresRMSE[j])])

                csvfile.writerow(["RHS: {}".format(cv_tempScoresRHS[j])])

                csvfile.writerow(["Time : {} seconds".format(cv_tempTime[j])])


                print("Test set")
                tModel.evaluate_model(['rhs', 'rmse'], round=2, cross_validation=False)

                cScores = tModel.scores
                rmse = math.sqrt(cScores['score_1'])
                rmse2 = cScores['rmse']
                rhs = cScores['rhs']
                time = tModel.train_time

                tempScoresRMSE[j] = rmse2
                tempScoresRHS[j] = rhs
                tempTime[j] = time

                n = range(1,len(tModel.y_crossVal)+1)


                for n, rul_prediction, rul_prediction_rounded, true_rul in zip(n, np.ravel(tModel.y_predicted), tModel.y_predicted_rounded, np.ravel(tModel.y_test)):
                    p = '{}, {}'.format(rul_prediction_rounded, true_rul)
                    print(p)
                    csvfile.writerow([p])
                #     p = "Engine {},Predicted RUL {},Rounded RUL {},Real RUL {}".format(i, rul_prediction, 
                #                                                                     rul_prediction_rounded, 
                #                                                                     true_rul)
                #     print(p)
                #     csvfile.writerow(["Engine {} Predicted RUL {} Rounded RUL {} Real RUL {}".format(i,rul_prediction, 
                #                                                                     rul_prediction_rounded, 
                #                                                                     true_rul)])
                print(cScores)
                #print("RMSE: {}".format(rmse))
                print("RMSE2: {}".format(tempScoresRMSE[j]))
                print("RHS: {}".format(tempScoresRHS[j]))
                print("Time : {} seconds".format(tempTime[j]))

                csvfile.writerow(["RMSE2: {}".format(tempScoresRMSE[j])])

                csvfile.writerow(["RHS: {}".format(tempScoresRHS[j])])

                csvfile.writerow(["Time : {} seconds".format(tempTime[j])])


    #             print("Results for cross validtion model " + key)

    #             print(stats.describe(cv_tempScoresRMSE))
    #             print(stats.describe(cv_tempScoresRHS))
    #             print(stats.describe(cv_tempTime))

    #             cv_tempScoresRMSE = np.reshape(cv_tempScoresRMSE, (iterations,))
    #             cv_tempScoresRHS = np.reshape(cv_tempScoresRHS, (iterations,))
    #             cv_tempTime = np.reshape(cv_tempTime, (iterations,))

    #             csvfile.writerow(cv_tempScoresRMSE)
    #             csvfile.writerow(cv_tempScoresRHS)
    #             csvfile.writerow(cv_tempTime)

    #             print("Results for test model " + key)

    #             print(stats.describe(tempScoresRMSE))
    #             print(stats.describe(tempScoresRHS))
    #             print(stats.describe(tempTime))

    #             tempScoresRMSE = np.reshape(tempScoresRMSE, (iterations,))
    #             tempScoresRHS = np.reshape(tempScoresRHS, (iterations,))
    #             tempTime = np.reshape(tempTime, (iterations,))


    #             csvfile.writerow(tempScoresRMSE)
    #             csvfile.writerow(tempScoresRHS)
    #             csvfile.writerow(tempTime)


        
file.close()

For model lstm-20-10
Computing for dataset 1
Loading data for dataset 1 with window_size of 30, stride of 1 and maxRUL of 125. Cros-Validation ratio 0.2
Loading data from file and computing dataframes


/home/controlslab/anaconda3/envs/tf_gpu/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/controlslab/anaconda3/envs/tf_gpu/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/media/controlslab/DATA/Projects/NASA_RUL_-CMAPS-/code/data_handler_CMAPS.py:133: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  norm_test_df = pd.DataFrame(self._data_scaler.transform(self._df_test[cols_normalize]), columns=cols_normalize, index=self._df_test.index)


cv number
[39, 47, 26, 52, 25, 90, 88, 78, 35, 56, 8, 87, 97, 69, 75, 74, 12, 32, 57, 42]
training with cv
CV set
20/20 [==============================] - 0s 90us/step
117.0, 92.0
59.0, 50.0
114.0, 116.0
97.0, 104.0
117.0, 148.0
117.0, 87.0
6.0, 6.0
97.0, 127.0
5.0, 4.0
61.0, 109.0
4.0, 10.0
119.0, 90.0
120.0, 199.0
44.0, 26.0
91.0, 96.0
33.0, 45.0
72.0, 84.0
89.0, 69.0
19.0, 17.0
74.0, 53.0
{'loss': 713.9918212890625, 'score_1': 713.9918212890625, 'rhs': 28.27662629922358, 'rmse': 26.85609800399157}
RMSE2: [26.856098]
RHS: [28.2766263]
Time : [23.333751] seconds
Test set
10/10 [==============================] - 0s 229us/step
121.0, 112.0
119.0, 98.0
32.0, 69.0
80.0, 82.0
119.0, 91.0
104.0, 93.0
119.0, 91.0
119.0, 95.0
106.0, 111.0
105.0, 96.0
{'loss': 433.50543212890625, 'score_1': 433.50543212890625, 'rhs': 6.985879079234148, 'rmse': 20.654297373670207}
RMSE2: [20.65429737]
RHS: [6.98587908]
Time : [23.333751] seconds
Loading data for dataset 1 with window_size of 30, stride of 1 and

114.0, 112.0
116.0, 98.0
40.0, 69.0
71.0, 82.0
105.0, 91.0
105.0, 93.0
94.0, 91.0
87.0, 95.0
116.0, 111.0
100.0, 96.0
{'loss': 177.818115234375, 'score_1': 177.818115234375, 'rhs': 2.2624828098711314, 'rmse': 13.206059215375342}
RMSE2: [13.20605922]
RHS: [2.26248281]
Time : [21.698178] seconds
Loading data for dataset 1 with window_size of 30, stride of 1 and maxRUL of 125. Cros-Validation ratio 0.2
Loading data from memory without recomputing df
cv number
[61, 73, 81, 28, 13, 52, 83, 96, 74, 69, 29, 93, 86, 99, 10, 45, 94, 71, 9, 75]
training with cv
CV set
20/20 [==============================] - 0s 96us/step
2.0, 0.0
81.0, 94.0
82.0, 86.0
118.0, 96.0
3.0, 3.0
120.0, 87.0
117.0, 180.0
120.0, 118.0
110.0, 183.0
3.0, 1.0
63.0, 151.0
119.0, 126.0
20.0, 32.0
37.0, 49.0
115.0, 104.0
99.0, 75.0
95.0, 101.0
119.0, 145.0
110.0, 229.0
110.0, 136.0
{'loss': 1744.5738525390625, 'score_1': 1744.5738525390625, 'rhs': 539.2625106442263, 'rmse': 42.068396689201265}
RMSE2: [42.06839669]
RHS: [539.26

119.0, 112.0
121.0, 98.0
34.0, 69.0
90.0, 82.0
102.0, 91.0
88.0, 93.0
108.0, 91.0
112.0, 95.0
111.0, 111.0
119.0, 96.0
{'loss': 323.96453857421875, 'score_1': 323.96453857421875, 'rhs': 4.5374479899033355, 'rmse': 17.663521732655695}
RMSE2: [17.66352173]
RHS: [4.53744799]
Time : [20.296443] seconds
Loading data for dataset 1 with window_size of 30, stride of 1 and maxRUL of 125. Cros-Validation ratio 0.30000000000000004
Loading data from memory without recomputing df
cv number
[53, 20, 68, 91, 58, 46, 95, 70, 92, 52, 60, 81, 63, 15, 51, 62, 3, 65, 99, 59, 56, 34, 84, 67, 90, 1, 8, 35, 83, 93]
training with cv
CV set
30/30 [==============================] - 0s 54us/step
26.0, 30.0
121.0, 124.0
83.0, 66.0
99.0, 101.0
60.0, 60.0
122.0, 158.0
111.0, 80.0
116.0, 184.0
116.0, 103.0
120.0, 123.0
53.0, 66.0
118.0, 145.0
4.0, 3.0
120.0, 103.0
30.0, 31.0
35.0, 37.0
121.0, 135.0
65.0, 64.0
79.0, 180.0
123.0, 161.0
120.0, 99.0
122.0, 135.0
85.0, 116.0
12.0, 16.0
67.0, 56.0
43.0, 39.0
8.0, 17.0
74.

...

training with cv
CV set
50/50 [==============================] - 0s 76us/step
121.0, 250.0
119.0, 137.0
116.0, 110.0
39.0, 51.0
2.0, 0.0
108.0, 115.0
119.0, 160.0
75.0, 83.0
66.0, 75.0
12.0, 15.0
113.0, 217.0
16.0, 14.0
17.0, 16.0
26.0, 37.0
114.0, 177.0
102.0, 76.0
120.0, 171.0
44.0, 43.0
120.0, 164.0
107.0, 111.0
115.0, 103.0
119.0, 122.0
60.0, 65.0
100.0, 76.0
85.0, 96.0
115.0, 140.0
78.0, 69.0
58.0, 70.0
112.0, 95.0
120.0, 122.0
4.0, 6.0
25.0, 20.0
112.0, 83.0
106.0, 87.0
80.0, 121.0
118.0, 161.0
2.0, 6.0
69.0, 64.0
112.0, 171.0
81.0, 150.0
63.0, 58.0
121.0, 104.0
100.0, 95.0
54.0, 75.0
117.0, 90.0
77.0, 74.0
113.0, 162.0
23.0, 20.0
114.0, 83.0
26.0, 28.0
{'loss': 1175.5354248046874, 'score_1': 1175.5354248046874, 'rhs': 482.1424808563638, 'rmse': 34.38866092187946}
RMSE2: [34.38866092]
RHS: [482.14248086]
Time : [14.789481] seconds
Test set
10/10 [==============================] - 0s 156us/step
117.0, 112.0
117.0, 98.0
37.0, 69.0
87.0, 82.0
81.0, 91.0
85.0, 93.0
113.0, 91.0
112.0,

training with cv
CV set
60/60 [==============================] - 0s 67us/step
62.0, 64.0
110.0, 90.0
56.0, 47.0
117.0, 234.0
108.0, 172.0
30.0, 23.0
42.0, 39.0
119.0, 126.0
103.0, 104.0
118.0, 96.0
66.0, 97.0
101.0, 75.0
96.0, 124.0
26.0, 26.0
45.0, 63.0
4.0, 5.0
116.0, 128.0
103.0, 97.0
119.0, 96.0
118.0, 103.0
13.0, 13.0
103.0, 152.0
118.0, 76.0
77.0, 60.0
106.0, 120.0
105.0, 105.0
96.0, 118.0
65.0, 82.0
11.0, 8.0
103.0, 132.0
80.0, 85.0
65.0, 56.0
102.0, 156.0
38.0, 49.0
119.0, 97.0
71.0, 59.0
83.0, 89.0
111.0, 202.0
112.0, 80.0
109.0, 137.0
120.0, 330.0
32.0, 28.0
113.0, 165.0
76.0, 94.0
120.0, 168.0
52.0, 40.0
4.0, 0.0
119.0, 147.0
67.0, 55.0
43.0, 36.0
103.0, 125.0
29.0, 26.0
86.0, 138.0
83.0, 56.0
17.0, 22.0
95.0, 112.0
92.0, 113.0
119.0, 161.0
116.0, 152.0
36.0, 33.0
{'loss': 1672.6130045572916, 'score_1': 1672.6130045572916, 'rhs': 172897.31316252894, 'rmse': 41.037787464725724}
RMSE2: [41.03778746]
RHS: [172897.31316253]
Time : [11.131093] seconds
Test set
10/10 [============

training with cv
CV set
60/60 [==============================] - 0s 67us/step
119.0, 176.0
119.0, 107.0
106.0, 154.0
120.0, 176.0
109.0, 140.0
3.0, 7.0
118.0, 137.0
118.0, 207.0
30.0, 33.0
78.0, 131.0
118.0, 84.0
36.0, 67.0
20.0, 26.0
116.0, 160.0
110.0, 115.0
98.0, 168.0
77.0, 98.0
16.0, 15.0
119.0, 160.0
84.0, 69.0
102.0, 57.0
40.0, 40.0
107.0, 85.0
119.0, 119.0
116.0, 125.0
115.0, 183.0
119.0, 158.0
83.0, 49.0
82.0, 65.0
120.0, 180.0
4.0, 7.0
9.0, 11.0
119.0, 67.0
87.0, 154.0
113.0, 98.0
88.0, 61.0
76.0, 61.0
96.0, 105.0
120.0, 139.0
114.0, 126.0
116.0, 94.0
36.0, 46.0
72.0, 67.0
117.0, 155.0
118.0, 90.0
116.0, 130.0
6.0, 10.0
26.0, 26.0
118.0, 222.0
119.0, 115.0
60.0, 62.0
4.0, 0.0
67.0, 81.0
49.0, 73.0
120.0, 95.0
70.0, 115.0
119.0, 230.0
54.0, 43.0
114.0, 80.0
57.0, 108.0
{'loss': 1467.8765869140625, 'score_1': 1467.8765869140625, 'rhs': 176.64979770424438, 'rmse': 38.41549166677422}
RMSE2: [38.41549167]
RHS: [176.6497977]
Time : [11.803975] seconds
Test set
10/10 [==============

121.0, 112.0
121.0, 98.0
43.0, 69.0
90.0, 82.0
121.0, 91.0
73.0, 93.0
82.0, 91.0
115.0, 95.0
100.0, 111.0
66.0, 96.0
{'loss': 413.6438903808594, 'score_1': 413.6438903808594, 'rhs': 5.856058511981511, 'rmse': 20.376457003120045}
RMSE2: [20.376457]
RHS: [5.85605851]
Time : [9.756821] seconds
Loading data for dataset 1 with window_size of 30, stride of 1 and maxRUL of 125. Cros-Validation ratio 0.7
Loading data from memory without recomputing df
cv number
[8, 20, 27, 54, 66, 12, 62, 1, 49, 35, 71, 98, 90, 38, 46, 15, 86, 58, 92, 18, 26, 59, 13, 11, 95, 29, 100, 24, 63, 64, 36, 80, 16, 2, 85, 99, 60, 42, 65, 48, 53, 77, 43, 82, 93, 96, 45, 17, 55, 33, 31, 23, 51, 88, 75, 79, 61, 56, 57, 50, 10, 83, 32, 41, 84, 28, 72, 34, 91, 4]
training with cv
CV set
70/70 [==============================] - 0s 77us/step
116.0, 119.0
119.0, 237.0
112.0, 126.0
64.0, 59.0
95.0, 68.0
68.0, 70.0
78.0, 72.0
45.0, 44.0
100.0, 89.0
114.0, 160.0
1.0, 2.0
76.0, 75.0
106.0, 186.0
114.0, 94.0
106.0, 96.0
120.0, 164

training with cv
CV set
70/70 [==============================] - 0s 80us/step
114.0, 45.0
45.0, 43.0
2.0, 1.0
83.0, 121.0
27.0, 41.0
79.0, 115.0
17.0, 20.0
85.0, 89.0
26.0, 29.0
122.0, 135.0
46.0, 69.0
109.0, 83.0
116.0, 167.0
39.0, 47.0
3.0, 9.0
120.0, 115.0
69.0, 157.0
16.0, 13.0
112.0, 116.0
56.0, 55.0
26.0, 15.0
3.0, 2.0
113.0, 98.0
123.0, 188.0
71.0, 61.0
42.0, 30.0
107.0, 110.0
119.0, 112.0
72.0, 88.0
38.0, 31.0
18.0, 11.0
116.0, 183.0
4.0, 3.0
121.0, 142.0
72.0, 154.0
20.0, 23.0
7.0, 18.0
35.0, 47.0
122.0, 125.0
116.0, 66.0
101.0, 72.0
7.0, 9.0
122.0, 120.0
14.0, 24.0
39.0, 31.0
80.0, 123.0
64.0, 113.0
55.0, 62.0
41.0, 81.0
100.0, 88.0
47.0, 66.0
8.0, 1.0
120.0, 166.0
113.0, 84.0
120.0, 147.0
26.0, 29.0
38.0, 53.0
67.0, 74.0
43.0, 51.0
118.0, 211.0
39.0, 57.0
36.0, 50.0
120.0, 153.0
124.0, 107.0
92.0, 230.0
71.0, 154.0
58.0, 71.0
117.0, 299.0
46.0, 35.0
99.0, 117.0
{'loss': 1713.8467912946428, 'score_1': 1713.8467912946428, 'rhs': 17834.86709564329, 'rmse': 41.57420216569749}
RM

# Test on all Datasets

In [9]:
# datasets = [1,2,3,4]
# iterations = 10
# tModel.epochs = 150
# lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)
# scores ={1:[], 2:[], 3:[], 4:[]}
# window_sizes = {1:17,2:17,3:17,4:17}
# strides = {1:1,2:1,3:1,4:1}
# max_ruls = {1:129, 2:129, 3:129, 4:129}
# num_features = len(selected_features)

# input_shape = None
# models = {'shallow-20-20':RULmodel_SN_4}

# #For each model
# for key, model_def in models.items():
#     file = open("results/MLP/ResultsDatasets_singleSet"+key+".csv", "w")
#     csvfile = csv.writer(file, lineterminator='\n')
    
#     print("Generating statistics for model " + key)

#     #For each dataset
#     for i in range(1,5):
        
#         print("Working on dataset " + str(i))
        
#         tempScoresRMSE = np.zeros((iterations,1))
#         tempScoresRHS = np.zeros((iterations,1))
#         tempTime = np.zeros((iterations,1))
        
#         input_shape = window_sizes[i]*num_features #For simple ANN
#         #input_shape = (window_sizes[i],num_features) #For RNN
        
#         print(input_shape)
        
#         tModel.data_handler.change_dataset(i)
#         tModel.data_handler.sequence_length = window_sizes[i]
#         tModel.data_handler.sequence_stride = strides[i]
#         tModel.data_handler.max_rul = max_ruls[i]
#         tModel.load_data(unroll=True, verbose=0, cross_validation_ratio=0)
#         #tModel.print_data()
        
#         #tModel.print_data()
        
#         for j in range(iterations):

#             #Model needs to be recompiled everytime since they are different runs so weights should be reinit
#             model = get_compiled_model(model_def, input_shape, model_type='ann')

#             tModel.change_model(key, model, 'keras')
#             tModel.train_model(learningRate_scheduler=lrate, verbose=0)
#             tModel.evaluate_model(['rhs', 'rmse'], round=2)
#             #print("scores")
            
#             #print(j)

#             cScores = tModel.scores
#             rmse = math.sqrt(cScores['score_1'])
#             rmse2 = cScores['rmse']
#             rhs = cScores['rhs']
#             time = tModel.train_time
            
#             tempScoresRMSE[j] = rmse2
#             tempScoresRHS[j] = rhs
#             tempTime[j] = time
            
#         print("Results for model " + key)
    
#         print(stats.describe(tempScoresRMSE))
#         print(stats.describe(tempScoresRHS))
#         print(stats.describe(tempTime))
            
#         tempScoresRMSE = np.reshape(tempScoresRMSE, (iterations,))
#         tempScoresRHS = np.reshape(tempScoresRHS, (iterations,))
#         tempTime = np.reshape(tempTime, (iterations,))
#         csvfile.writerow(tempScoresRMSE)
#         csvfile.writerow(tempScoresRHS)
#         csvfile.writerow(tempTime)
    
#     file.close()